In [9]:
import pandas as pd
import os
from functools import reduce
import json


# Directory

In [10]:
### 현재 작업 디렉토리를 가져오기
current_directory = os.getcwd()

### downloads 상대경로를 절대경로로 변환
relative_path_downloads = r'..\\..\\..\\Downloads'
download_path = os.path.join(current_directory, relative_path_downloads)

### "preprocessed" 상대경로를 절대경로로 변환
preprocessed_path = os.path.join(download_path, 'preprocessed')

### Check if the "preprocessed" folder exists
if os.path.exists(preprocessed_path):
    print(f"The 'preprocessed' folder already exists at: {preprocessed_path}")
else:
    # "preprocessed" 폴더 생성
    os.makedirs(preprocessed_path)

### ContentsList 상대경로를 절대경로로 변환
relative_path_contentslist = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\001_ContentsList.xlsx'
contentslist_path = os.path.join(current_directory, relative_path_contentslist)

### DailyRawdata
path_dailyrawdata = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\01_DailyRawdata'


### 01_콘텐츠성과측정 상대경로를 절대경로로 변환
relative_path_upper_path = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정'
upper_path = os.path.join(current_directory, relative_path_upper_path)


### 02_ConvertedData 상대경로를 절대경로로 변환
relative_path_convert_path = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\02_ConvertedData'
convert_path = os.path.join(current_directory, relative_path_convert_path)

### new_id 파일 경로 설정
mapping_file_path = os.path.join(relative_path_convert_path, 'id_mapping.json')


### 03_CombinedData 상대경로를 절대경로로 변환
relative_path_combine_path = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\03_CombinedData'
combine_path = os.path.join(current_directory, relative_path_combine_path)


### 04_RequestedData 상대경로를 절대경로로 변환
relative_path_request_path = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\04_RequestedData'
request_path = os.path.join(current_directory, relative_path_request_path)




The 'preprocessed' folder already exists at: c:\Users\ch.kang\Documents\GitHub\funnel_test\..\\..\\..\\Downloads\preprocessed


In [11]:
# 폴더 내의 파일 목록 가져오기
file_list = os.listdir(path_dailyrawdata)

# 'ga'로 시작하는 전체 파일 필터링
ga_files = [file for file in file_list if file.startswith('ga')]

# 'ga'로 시작하는 날짜별 파일 필터링: 주석에 유의하자
# 날짜 범위
# start_date = datetime(2024, 3, 22)
# end_date = datetime(2024, 3, 24)
#ga_files = [file for file in file_list if file.startswith('ga') and start_date <= datetime.strptime(file.split('_')[1].split('.')[0], '%Y%m%d') <= end_date]

# 모든 '아이디' 값들을 저장할 빈 DataFrame 생성
all_ids = pd.DataFrame()

for file in ga_files:
    file_path = os.path.join(path_dailyrawdata, file)
    temp_df = pd.read_excel(file_path, dtype=str)
    
    # 현재 파일에서 '아이디' 열만 추출하여 all_ids에 추가
    current_ids = temp_df[['아이디']].copy()  # '아이디' 열만 선택
    all_ids = pd.concat([all_ids, current_ids], ignore_index=True)

# 중복 제거
all_ids_unique = all_ids.drop_duplicates().reset_index(drop=True)
all_ids_unique['new_id'] = ['g-' + f'{i:06d}' for i in range(1, len(all_ids_unique) + 1)]
all_ids_unique.rename(columns={'아이디': 'pseudo_id'}, inplace=True)
# 매핑 정보 파일 경로
all_ids_unique.to_csv(mapping_file_path, index=False)
# 결과 확인
print(all_ids_unique)

# 새로운 '아이디' 업데이트 함수
def update_id_mapping(existing_mapping_df, new_ids):
    if not existing_mapping_df.empty:
        max_new_id = max(existing_mapping_df['new_id'].apply(lambda x: int(x.split('-')[1])))+1
    else:
        max_new_id = 1
    new_rows = []
    for new_id in new_ids:
        if new_id not in existing_mapping_df['pseudo_id'].values:
            new_row = {'pseudo_id': new_id, 'new_id': f'g-{max_new_id:06d}'}
            new_rows.append(new_row)
            max_new_id += 1
    if new_rows:
        updated_df = pd.concat([existing_mapping_df, pd.DataFrame(new_rows)], ignore_index=True)
    else:
        updated_df = existing_mapping_df
    return updated_df

# 매핑 정보 저장 함수
def save_id_mapping(mapping_df, mapping_file_path):
    mapping_df.to_csv(mapping_file_path, index=False)




# Convert GA Data

In [12]:
for file in ga_files:
    # 파일 이름과 확장자 분리 및 "_pivot" 접미사 추가
    file_name, file_ext = os.path.splitext(file)
    file_name_with_suffix = file_name + "_pivot"
    new_file_name = file_name_with_suffix + file_ext

    # 최종 파일 경로 생성
    file_path_convert = os.path.join(convert_path, new_file_name)

    # 만약 변환된 파일이 이미 존재한다면, 다음 파일로 넘어감
    if os.path.exists(file_path_convert):
        print(f"File already exists: {file_path_convert}")
        continue  # 다음 파일로 넘어감

    file_path = os.path.join(path_dailyrawdata, file)  # 파일의 전체 경로 생성
    temp_df = pd.read_excel(file_path, dtype=str) # 파일을 데이터프레임으로 읽기

    # # new_id 업데이트
    # ids = set(temp_df['아이디'].unique())  # 고유 '아이디' 추출
    # id_mapping_df = update_id_mapping(id_mapping_df, list(ids))  # 집합을 리스트로 변환하여 함수에 전달
    # save_id_mapping(id_mapping_df, mapping_file_path)
    # 데이터프레임 로드

    # 고유 아이디 매핑 파일 로드
    with open(mapping_file_path, 'r') as f:
        id_mapping = json.load(f)

    # UniqueIDGenerator 클래스
    class UniqueIDGenerator:
        def __init__(self, id_mapping):
            self.id_mapping = id_mapping

        def get_unique_id(self, username):
            return self.id_mapping.get(username, None)

    # UniqueIDGenerator 인스턴스 생성
    generator = UniqueIDGenerator(id_mapping)

    # 고유 아이디 생성 및 데이터프레임에 추가
    temp_df['고유아이디'] = temp_df['아이디'].apply(generator.get_unique_id)


    # 파일처리 시작!!! (admin 처리 -> 기본정보 / 전환정보 / depth 정보 구분 -> pivot and merge)
    df = temp_df

    # 시간 -> date, time으로 분리 // header 영어 지정 // new_id 매핑
    df['date'] = df['시간'].str[:-11]  # 뒤에서 11번째 문자까지가 날짜
    df['time'] = df['시간'].str[-11:]  # 뒤에서 10자리가 시간
    df = df.drop('시간', axis=1)    # "시간" 열 제거
    df = df.rename(columns={
        '아이디': 'pseudo_id',
        '고유아이디': 'new_id',
        '이벤트명': 'event_name',
        '페이지제목': 'page_title',
        '정규화링크': 'normalized_link',
        '원래링크': 'original_link',
        '소스': 'source',
        '매체': 'medium',
        '캠페인': 'campaign',
        '컨텐츠': 'content',
        '검색어': 'keyword',
        '체류시간': 'duration',
        '전환여부': 'conversion',
        'date': 'date',
        'time': 'time'
    })

    # admin id 처리
    condition_a = df['normalized_link'].str.contains('write|prod_code', case=False, regex=True)
    condition_a = condition_a.fillna(False)  # NaN 값을 False로 대체
    condition_a.value_counts()
    filtered_a = df[condition_a]
    ids_a = filtered_a['pseudo_id'].unique().tolist()  # 기존의 고유한 아이디 목록을 리스트로 변환
    ids_a.extend(['1828185651.1705389498', '417241095.1705634348', '1602359817.1707964923'])  # 새로운 아이디를 추가
    ids_b = ['1021566177.1707466356', '1083942841.1695711394',
        '1120703410.1706587025', '112167875.1704799413',
        '1140615326.1706539096', '1158917536.1707880810',
        '1188470945.1704806360', '1195499539.1705461693',
        '1208982378.1707455391', '1250133156.1706801009',
        '130036583.1660629517', '1302147567.1707929419',
        '1312396219.1707191882', '1312429530.1677388875',
        '1323533573.1707880592', '1341286132.1689068932',
        '1341500281.1707228435', '135745452.1687348999',
        '1362795537.1707364318', '1392746774.1707093754',
        '1414598046.1706604478', '1437195581.1701747873',
        '1535161370.1707530975', '1542469471.1706330459',
        '1549941021.1706832674', '1588483927.1696296964',
        '1592112601.1686896453', '1627436795.1707440156',
        '1644808335.1706455834', '1645381003.1707756395',
        '1739828001.1705369497', '1757415153.1705996376',
        '1771636829.1707531355', '198483479.1707868974',
        '2054491282.1707951088', 
        '2078435867.1706074815', '2105957382.1707103400',
        '2115567724.1705386386', '278436795.1700370232', '288263341.1705334410',
        '302251614.1707783216', '306548010.1699417096', '335460286.1707105953',
        '352146708.1707529167', '400501807.1707570514', '420765241.1705305938',
        '48543513.1707440466', '52044247.1693770278', '54599874.1707924934',
        '555071005.1702130820', '577908767.1706330356', '593811738.1706411252',
        '605077705.1702539997', '639043677.1706935086', '649116625.1707636536',
        '736497018.1707340441', '740700171.1705212161', '744027507.1707038633',
        '768492148.1699924210', '771137756.1707490369', '799231654.1707925045',
        '813986486.1707308094', '838526411.1705675392', '858944222.1699852150',
        '861783116.1705336795', '892793016.1701953466', '910079147.1700545452',
        '920629470.1705452306', '936659568.1707636674', '965385222.1704683076',
        '97146443.1707440532']
    admin_ids = set(ids_a) | set(ids_b)
    df_with_new_id_noadmin= df[~df['pseudo_id'].isin(admin_ids)]


    # 기본정보
    basic_info_columns = ['new_id', 'pseudo_id', 'date', 'time', 'source', 'medium', 'campaign', 'content', 'keyword']
    df_basic_info = df_with_new_id_noadmin.sort_values(by=['new_id', 'time']).drop_duplicates(subset='new_id', keep='first')[basic_info_columns]


    # 기본정보 제외 -> 피벗
    exclude_columns = basic_info_columns  
    remaining_columns = [col for col in df.columns if col not in exclude_columns and col != 'pseudo_id']
    remaining_columns = ['new_id'] + remaining_columns
    remaining_info_df = df[remaining_columns]
    remaining_info_columns = remaining_columns[1:]  # '아이디'를 제외한 나머지 열

    pivot_data_frames = []     # 아이디별로 각 나머지 정보 열의 값에 대해 피벗하기 위한 준비


    # 각 열에 대해 아이디별로 중복된 값을 처리하고 각각의 값을 고유한 열로 펼치기 위해 각 열별로 작업을 수행
    for col in remaining_info_columns:
        # 특정 열과 'new_id' 열만 추출
        temp_df = remaining_info_df[['new_id', col]].dropna()
        
        # pivot 작업: 각 아이디별로 열의 값을 순서대로 나열
        pivot_df = temp_df.pivot_table(index='new_id', columns=temp_df.groupby('new_id').cumcount()+1, values=col, aggfunc='first')
        pivot_df.columns = [f'{int(i):03d}+{col}' for i in pivot_df.columns]  # 세 자리 숫자로 포맷팅
        
        pivot_data_frames.append(pivot_df.reset_index())

    df_pivoted_remaining_info = reduce(lambda left, right: pd.merge(left, right, on='new_id', how='outer'), pivot_data_frames)     # 모든 피벗된 데이터프레임을 아이디를 기준으로 합치기

    # df_basic_info, df_pivoted_remaining_info를 new_id를 기준으로 합치기
    df_basic_info.columns = ['000+' + col if col != 'new_id' else col for col in df_basic_info.columns]

    # df_merged를 두 레벨의 멀티인덱스로 만들기
    df_merged = pd.merge(df_basic_info, df_pivoted_remaining_info, on='new_id', how='outer')
    df_merged.rename(columns={'new_id': '0+new_id'}, inplace=True)
    column_names = df_merged.columns.tolist()    # 열 이름을 문자열 리스트로 변환
    split_columns = [name.split('+') if '+' in name else [name, None] for name in column_names]     # '+' 기준으로 분할하고, 빈 문자열을 None으로 변환
    df_merged.columns = pd.MultiIndex.from_tuples(split_columns)     # None을 포함한 분할 결과를 멀티 인덱스로 변환
    df_sorted = df_merged.sort_index(axis=1, level=0)

    # 출력, 저장
    df_merged.to_excel(file_path_convert)
    print('file:', file, '//1.최초 행:', len(df), '//2. 정리한 행',len(df_sorted))




file: ga_20240115.xlsx //1.최초 행: 594 //2. 정리한 행 83
file: ga_20240116.xlsx //1.최초 행: 697 //2. 정리한 행 121


C:\Users\ch.kang\AppData\Local\Temp\ipykernel_27128\4092874258.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  condition_a = condition_a.fillna(False)  # NaN 값을 False로 대체


file: ga_20240117.xlsx //1.최초 행: 942 //2. 정리한 행 164
file: ga_20240118.xlsx //1.최초 행: 760 //2. 정리한 행 146


C:\Users\ch.kang\AppData\Local\Temp\ipykernel_27128\4092874258.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  condition_a = condition_a.fillna(False)  # NaN 값을 False로 대체


file: ga_20240119.xlsx //1.최초 행: 916 //2. 정리한 행 139


...

file: ga_20240414.xlsx //1.최초 행: 3624 //2. 정리한 행 1089
file: ga_20240415.xlsx //1.최초 행: 3684 //2. 정리한 행 979
file: ga_20240416.xlsx //1.최초 행: 4766 //2. 정리한 행 1008
file: ga_20240417.xlsx //1.최초 행: 3730 //2. 정리한 행 992
file: ga_20240418.xlsx //1.최초 행: 4018 //2. 정리한 행 1028
file: ga_20240419.xlsx //1.최초 행: 1462 //2. 정리한 행 295
file: ga_20240420.xlsx //1.최초 행: 1722 //2. 정리한 행 317
file: ga_20240421.xlsx //1.최초 행: 516 //2. 정리한 행 132
file: ga_20240422.xlsx //1.최초 행: 1083 //2. 정리한 행 151


C:\Users\ch.kang\AppData\Local\Temp\ipykernel_27128\4092874258.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  condition_a = condition_a.fillna(False)  # NaN 값을 False로 대체


file: ga_20240423.xlsx //1.최초 행: 1042 //2. 정리한 행 164


C:\Users\ch.kang\AppData\Local\Temp\ipykernel_27128\4092874258.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  condition_a = condition_a.fillna(False)  # NaN 값을 False로 대체


file: ga_20240424.xlsx //1.최초 행: 1578 //2. 정리한 행 175
